# Enum Q&A

Python enums are simple on the surface, but packs some power underneath which makes it easier to use.

## How do I create an enum?

There are two ways to create an enum, the first is to sub-class `enum.Enum`:

In [1]:
import enum

class BrowserType(enum.Enum):
    CHROME = "chrome"
    SAFARI = "safari"
    
print(BrowserType.CHROME)

BrowserType.CHROME


Note some terminology: 

- `BrowserType.CHROME` is a member
- `CHROME` is called a name
- `"chrome"` is a value.

The second is to use `enum.Enum` as a function:

In [2]:
import enum

StreamingProtocol = enum.Enum("StreamingProtocol", "HLS DASH")
print(StreamingProtocol.HLS)
print(StreamingProtocol.DASH)

StreamingProtocol.HLS
StreamingProtocol.DASH


## How do I convert from a string value to an enum?

Given the `BrowserType` enum, how do I convert from the string `"chrome"` to `BrowserType.CHROME`? The answer is easy:

In [3]:
BrowserType("chrome")

<BrowserType.CHROME: 'chrome'>

## Given an enum, how do I get its value?
That is, given `BrowserType.SAFARI`, I want to get `"safari"`

In [4]:
BrowserType.SAFARI.value

'safari'

## How do I convert from a string name to an enum?

Given a string `"CHROME"`, how do I convert it to `BrowserType.CHROME`?

In [5]:
BrowserType["CHROME"]

<BrowserType.CHROME: 'chrome'>

## How do I get a list of names?

An enum has an attribute `__members__` which is a mapping `{name: enum}`:

In [6]:
BrowserType.__members__

mappingproxy({'CHROME': <BrowserType.CHROME: 'chrome'>,
              'SAFARI': <BrowserType.SAFARI: 'safari'>})

To get a a list of names, convert that into a list:

In [7]:
list(BrowserType.__members__)

['CHROME', 'SAFARI']

## How do I get a list of values?

In [8]:
[e.value for e in BrowserType]

['chrome', 'safari']

## How do I iterate over all instances of an enum?

In [10]:
for browser in BrowserType:
    print(browser)

BrowserType.CHROME
BrowserType.SAFARI


## Aliases

Alias happens when we have many members which share the same value. Consider the following:

In [11]:
class WeekDay(enum.StrEnum):
    MON = "Monday"
    TUE = "Tuesday"
    WED = "Wednesday"
    THU = "Thursday"
    FRI = "Friday"
    SAT = "Saturday"
    SUN = "Sunday"
    MONDAY = "Monday"
    TUESDAY = "Tuesday"
    WEDNESDAY = "Wednesday"
    THURSDAY = "Thursday"
    FRIDAY = "Friday"
    SATURDAY = "Saturday"
    SUNDAY = "Sunday"

In this case, `WeekDay.MONDAY` is an alias for `WeekDay.MON`. Note that when we iterate through an enum, aliases are not listed:

In [12]:
list(WeekDay)

[<WeekDay.MON: 'Monday'>,
 <WeekDay.TUE: 'Tuesday'>,
 <WeekDay.WED: 'Wednesday'>,
 <WeekDay.THU: 'Thursday'>,
 <WeekDay.FRI: 'Friday'>,
 <WeekDay.SAT: 'Saturday'>,
 <WeekDay.SUN: 'Sunday'>]

If we want to iterate through both members and aliases:

In [14]:
WeekDay.__members__

mappingproxy({'MON': <WeekDay.MON: 'Monday'>,
              'TUE': <WeekDay.TUE: 'Tuesday'>,
              'WED': <WeekDay.WED: 'Wednesday'>,
              'THU': <WeekDay.THU: 'Thursday'>,
              'FRI': <WeekDay.FRI: 'Friday'>,
              'SAT': <WeekDay.SAT: 'Saturday'>,
              'SUN': <WeekDay.SUN: 'Sunday'>,
              'MONDAY': <WeekDay.MON: 'Monday'>,
              'TUESDAY': <WeekDay.TUE: 'Tuesday'>,
              'WEDNESDAY': <WeekDay.WED: 'Wednesday'>,
              'THURSDAY': <WeekDay.THU: 'Thursday'>,
              'FRIDAY': <WeekDay.FRI: 'Friday'>,
              'SATURDAY': <WeekDay.SAT: 'Saturday'>,
              'SUNDAY': <WeekDay.SUN: 'Sunday'>})

When we call an enum, it returns the member, not alias:

In [15]:
WeekDay('Monday')

<WeekDay.MON: 'Monday'>

## Given a case-insensitive value, how can we return a member?

In [17]:
# This will fail
try:
    WeekDay('monday')
except ValueError as error:
    print(error)

'monday' is not a valid WeekDay


The solution is to create a method named `_missing_()`, which gets called when value look-up failed:

In [18]:
class WeekDay(enum.StrEnum):
    MON = "Monday"
    TUE = "Tuesday"
    WED = "Wednesday"
    THU = "Thursday"
    FRI = "Friday"
    SAT = "Saturday"
    SUN = "Sunday"
    MONDAY = "Monday"
    TUESDAY = "Tuesday"
    WEDNESDAY = "Wednesday"
    THURSDAY = "Thursday"
    FRIDAY = "Friday"
    SATURDAY = "Saturday"
    SUNDAY = "Sunday"

    @classmethod
    def _missing_(cls, value: str):
        return cls(value.title())

In [19]:
# This still works
WeekDay("Monday")

<WeekDay.MON: 'Monday'>

In [20]:
# This now works, too.
WeekDay('monday')

<WeekDay.MON: 'Monday'>

## Given case insensitive name, how can we return a member?

In this case, we should implement a case-insensitive meta class for the enum:

In [81]:
import contextlib

class CaseInsensitiveKey(enum.EnumMeta):
    def __getitem__(self, name: str):
        try:
            return super().__getitem__(name)
        except KeyError:
            target = name.casefold()
            for name, member in self.__members__.items():
                if name.casefold() == target:
                    return member
            raise

class Color(enum.IntEnum, metaclass=CaseInsensitiveKey):
    Red = 1
    Green = 2
    Blue = 3
    CornFlowerBlue = 4
    Bleu = 3  # Alias for Blue

# Test it out
for name in ['red', 'cornflowerblue', 'BLUE', 'bleu', 'GReeN']:
    print(f"{name=} => {Color[name]!r}")

name='red' => <Color.Red: 1>
name='cornflowerblue' => <Color.CornFlowerBlue: 4>
name='BLUE' => <Color.Blue: 3>
name='bleu' => <Color.Blue: 3>
name='GReeN' => <Color.Green: 2>


This meta class `CaseInsensitiveKey` can be reused for other classes as well:

In [82]:
class PrinterType(enum.Enum, metaclass=CaseInsensitiveKey):
    UNKNOWN = 0
    MONOCHROME = 1
    COLOR = 2

# Test
for name in ['unknown', 'MonoChrome', 'COLOR']:
    print(f"{name=} => {PrinterType[name]!r}")

name='unknown' => <PrinterType.UNKNOWN: 0>
name='MonoChrome' => <PrinterType.MONOCHROME: 1>
name='COLOR' => <PrinterType.COLOR: 2>
